# Optimizing Pipelines Project

## Objective
This project is part of the Udacity Azure ML Nanodegree.  
In this project, we build and optimize an Azure ML pipeline using the Python SDK and a provided Scikit-learn model.  
This model is then compared to an Azure AutoML run.  

## 1. Hyperparameter run

### Initialize workspace and experiment

In [1]:
from azureml.core import Workspace, Experiment

#ws = Workspac<<e.get(name="udacity-project")
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-125880
Azure region: southcentralus
Subscription id: da775cb9-9ca6-4943-ad21-26dfa99526fc
Resource group: aml-quickstarts-125880


### Create Cluster

In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

#- Define cluster name
cluster_name = "compute-cluster"

#- Define cluster configuration
cluster_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', 
                                                       max_nodes=4)

#- Create cluster
compute_target = ComputeTarget.create(ws, cluster_name, cluster_config)

#- Set timeout
compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

#- Get cluster status
print(compute_target.get_status().serialize())

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2020-11-13T00:03:09.786000+00:00', 'errors': None, 'creationTime': '2020-11-13T00:03:07.080006+00:00', 'modifiedTime': '2020-11-13T00:03:22.806183+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


### Create environment

In [3]:
%%writefile conda_dependencies.yml

dependencies:
- python=3.6.2
- scikit-learn
- pip:
  - azureml-defaults

Writing conda_dependencies.yml


In [4]:
from azureml.core import Environment

sklearn_env = Environment.from_conda_specification(name = 'sklearn-env', file_path = './conda_dependencies.yml')

### Define hyperparameter space

In [5]:
#- Import libraries
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

#- These libraries were added
import shutil
from azureml.core import ScriptRunConfig
from azureml.train.hyperdrive import choice

In [6]:
# Specify parameter sampler
#ps = ### YOUR CODE HERE ###
ps = RandomParameterSampling(
    {
        '--C': choice(0.01, 0.1, 1.0, 10.0, 100.0),
        '--max_iter': choice(5000, 10000, 15000)
    }
)

In [7]:
# Specify a Policy
#policy = ### YOUR CODE HERE ###
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

In [8]:
#- Create project folder
if "training" not in os.listdir():
    os.mkdir("./training")

In [9]:
# Create a SKLearn estimator for use with train.py
#est = ### YOUR CODE HERE ###

#- Create project folder variable
project_folder = "./training"

#- Copy train.py file to project folder
shutil.copy('train.py', project_folder)

#- Create estimator
est = ScriptRunConfig(source_directory=project_folder,
                      script='train.py',
                      compute_target=compute_target,
                      environment=sklearn_env)

In [10]:
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
#hyperdrive_config = ### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(run_config=est,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4)

### Submit and monitor hyperdrive runs

In [11]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
#- Launch hyperparameter tuning job
hyperdrive_run = exp.submit(config=hyperdrive_config)

In [12]:
#- Show details
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [13]:
#- Wait for completion
hyperdrive_run.wait_for_completion(show_output=True)
assert(hyperdrive_run.get_status() == "Completed")

RunId: HD_62b0f300-8279-406e-97b4-e6a4cf4bff89
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_62b0f300-8279-406e-97b4-e6a4cf4bff89?wsid=/subscriptions/da775cb9-9ca6-4943-ad21-26dfa99526fc/resourcegroups/aml-quickstarts-125880/workspaces/quick-starts-ws-125880

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-11-13T00:03:54.127969][API][INFO]Experiment created<END>\n""<START>[2020-11-13T00:03:54.897144][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2020-11-13T00:03:55.185309][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2020-11-13T00:03:56.0187788Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_62b0f300-8279-406e-97b4-e6a4cf4bff89
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_62b0f300-8279-406e-97b4-e6a4cf4bff89?wsid=/subscriptions/da775

### Images of the hyperparameter runs

![Hyperparameter results 01](images/H_01.png)
![Hyperparameter results 02](images/H_02.png)
![Hyperparameter results 03](images/Hyperparameter_01.png)
![Hyperparameter results 04](images/Hyperparameter_02.png)

### Find and register best model

In [14]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
#- Get best run
best_run_h = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run_h.get_details()['runDefinition']['arguments'])

['--C', '10', '--max_iter', '10000']


In [15]:
#- List model files uploaded
print(best_run_h.get_file_names())

['azureml-logs/55_azureml-execution-tvmps_ea915f3a699258fc3501b64ca47a6668d6243a4f19f00c5d352d644028ece57b_d.txt', 'azureml-logs/65_job_prep-tvmps_ea915f3a699258fc3501b64ca47a6668d6243a4f19f00c5d352d644028ece57b_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_ea915f3a699258fc3501b64ca47a6668d6243a4f19f00c5d352d644028ece57b_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/100_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/engine_spans_l_cb614c2e-003e-4203-a722-5c51ac2a76d8.jsonl', 'logs/azureml/dataprep/python_span_l_cb614c2e-003e-4203-a722-5c51ac2a76d8.jsonl', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/model_h/model_h.joblib']


In [16]:
#- Download saved model
os.makedirs('./model_h', exist_ok=True)

for f in best_run_h.get_file_names():
    if f.startswith('outputs/model_h'):
        output_file_path = os.path.join('./model_h', f.split('/')[-1])
        print('Downloading from {} to {} ...'.format(f, output_file_path))
        best_run_h.download_file(name=f, output_file_path=output_file_path)

### Cluster clean up

In [17]:
#- Delete compute target
compute_target.delete()